In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 19.3 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [10]:
#%pip install google-cloud-bigquery
from google.cloud import bigquery
import logging
from datetime import datetime, timedelta

In [3]:
client = 'incrediwear@dataspeaks.io'

# BigQuery data
project_id = 'ageless-math-320621'
dataset_id = 'Models'
channel_level_table = 'MMM Aggregated'
campaign_level_table = 'MMM Campaigns'
platform_level_table = 'MMM Platform'

# Media / Control / Target strings to define variables
media_str = 'Ads'
organic_strs = 'organic'
control_str = 'Promotion'
web_str = '_web'
amazon_str = '_Amazon'
all_sales_str = 'Sales_'

In [4]:
gcp_project_id: str = "ageless-math-320621"
bq_client = bigquery.Client(project=gcp_project_id)

In [57]:
def get_df_from_bq(
    bq_client: bigquery.Client,
    project_id: str,
    dataset_id: str,
    table_name: str,
    client: str,
) -> pd.DataFrame:
    """Retrieve data from a BigQuery table and return it as a pandas DataFrame.

    This function executes a SQL query to select all records from the specified BigQuery table,
    filtered by a given username. It converts the result into a pandas DataFrame and the date column.

    Parameters:
        bq_client (bigquery.Client):
            The BigQuery client instance used to execute the query.
        project_id (str):
            The ID of the Google Cloud project containing the dataset.
        dataset_id (str):
            The ID of the dataset that contains the target table.
        table_name (str):
            The name of the table from which to retrieve data.
        client (str):
            The username used to filter the results.

    Returns:
        pd.DataFrame:
            A pandas DataFrame containing the query results with date columns converted
            to datetime and NaN values replaced with zero.

    Raises:
        ValueError:
            if there is an error when converting the column date to datetime or filling nan values
    """
    query = f'SELECT * FROM `{project_id}.{dataset_id}.{table_name}` WHERE Username = "{client}"'

    # Execute the query and convert the result to a pandas DataFrame
    df_from_bq = (
        bq_client.query(query)
        .result()
        .to_dataframe(create_bqstorage_client=True, progress_bar_type="tqdm")
    )
    try:
        if "Date" in df_from_bq.columns:
            df_from_bq["Date"] = pd.to_datetime(df_from_bq["Date"])
    except Exception as e:
        logging.error(
            f"Error converting 'Date' column from dbdatetime to datetime:\n {e}\n ",
            exc_info=True,
        )
        raise ValueError(
            f"Error converting 'Date' column from dbdatetime to datetime: \n {e}\n "
        ) from e

    try:
        df_from_bq = df_from_bq.apply(
            lambda col: col.replace(np.nan, 0) if col.dtype != "datetime64[ns]" else col
        )
    except Exception as e:
        logging.error(f"Error filling NaN values: \n {e}\n ", exc_info=True)
        raise ValueError(f"Error filling NaN values: {e}") from e

    return df_from_bq


def get_df_sliced_sorted(df: pd.DataFrame, years: float) -> pd.DataFrame:
    """Slice and sort a DataFrame based on date values.

    This function sorts the provided DataFrame by the 'Date' column in ascending order, converts
    the 'Date' column to datetime format, and filters the DataFrame to include only the records
    within the specified number of years from the current date.

    Parameters:
        df (pd.DataFrame):
            The input DataFrame containing a 'Date' column to be sliced and sorted.
        years (float):
            The number of years to look back from the current date.

    Returns:
        pd.DataFrame:
            A new DataFrame containing only the rows with the dates of the specified time window,
            sorted by the 'Date' column.
    """
    df = df.sort_values(by=["Date"], ascending=True)
    df["Date"] = pd.to_datetime(df["Date"])
    time_window_picked = datetime.now() - timedelta(days=years * 365)
    today = pd.to_datetime(datetime.now().strftime("%Y-%m-%d"))
    df = df[(df["Date"] > time_window_picked) & (df["Date"] < today)]

    return df

def cast_weekly_frequency(df_campaigns: pd.DataFrame) -> pd.DataFrame:
    """
    Convert daily campaign data into a weekly frequency DataFrame by summing numerical features
    and retaining the first username for each week.
    Parameters:
        df_campaigns (pd.DataFrame): A DataFrame containing campaign data with a
                                      'Date' column and various numerical columns
                                      along with a 'Username' column.

    Returns:
        pd.DataFrame: A new DataFrame indexed by week with the summed values of numerical
                       features and the first 'Username' for each week.
    """
    df_campaigns.set_index("Date", inplace=True)
    numerical_features = list(df_campaigns.select_dtypes(exclude="object").columns)
    df_campaign_weekly = df_campaigns[numerical_features].resample("W").sum()
    df_campaign_weekly["Username"] = df_campaigns["Username"].resample("W").first()

    return df_campaign_weekly

def drop_campaigns(df_campaigns: pd.DataFrame, strings_to_check: list) -> pd.DataFrame:
    """
    Remove specified columns from the campaign DataFrame based on provided strings
    and drop columns that contain all zero values
    Parameters:
        df_campaigns (pd.DataFrame): A DataFrame containing campaign data with
                                      various columns.
        strings_to_check (list): A list of strings to check for in the column names.
                                 Any column containing one of these strings will be dropped.

    Returns:
        pd.DataFrame: A cleaned DataFrame with specified columns removed and columns
                       with all zero values dropped.
    """
    cols_to_drop = [
        col
        for col in df_campaigns.columns
        if any(string in col for string in strings_to_check)
    ]
    df_campaigns_cleaned = df_campaigns.drop(columns=cols_to_drop)

    cols_all_zeros = df_campaigns_cleaned.eq(0).all(axis=0)
    df_campaigns_cleaned = df_campaigns_cleaned.drop(
        columns=cols_all_zeros[cols_all_zeros].index
    )

    return df_campaigns_cleaned


In [7]:
#%pip install db-dtypes==1.3.0

  Using cached db_dtypes-1.3.0-py2.py3-none-any.whl.metadata (3.0 kB)
Using cached db_dtypes-1.3.0-py2.py3-none-any.whl (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 3.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [68]:
df_campaigns_level = get_df_from_bq(bq_client= bq_client, project_id= project_id, dataset_id= dataset_id, table_name= campaign_level_table, client= client)
time = df_campaigns_level.Date
df_campaigns_level = get_df_sliced_sorted(df=df_campaigns_level, years=2.25)
df_campaign_weekly = cast_weekly_frequency(df_campaigns=df_campaigns_level)
strings_to_check = ['Sales_Amazon', 'Amazon_', 'Criteo_']
df_campaign_weekly_cleaned = drop_campaigns(df_campaigns=df_campaign_weekly, strings_to_check=strings_to_check)

/Users/fcremer29/focus/meridian_test/demo/env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/fcremer29/focus/meridian_test/demo/env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2508: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [61]:
df_campaign_weekly_cleaned.reset_index(inplace=True)

In [25]:
df_campaign_weekly_cleaned.columns

Index(['Facebook_Ads_Tests', 'Facebook_Ads_Retargeting',
       'Facebook_Ads_Advantage_Plus', 'TikTok_Ads_Nurturing',
       'Google_DSP_Ads_All', 'TikTok_Ads_Prospecting',
       'Facebook_Ads_Lead_generation', 'Google_Ads_Video',
       'Bing_Ads_Branded_search', 'Bing_Ads_Non_branded_search',
       'Bing_Ads_Shopping', 'Google_Ads_Demand_gen', 'Facebook_Ads_Nurturing',
       'Google_Ads_Shopping', 'Google_Ads_Non_branded_search',
       'Facebook_Ads_Promotions', 'Facebook_Ads_Prospecting',
       'Google_Ads_Branded_search', 'Google_Ads_Performance_Max',
       'TikTok_Ads_Product_ads', 'Sales_web', 'Promotion', 'Rank',
       'Google_Search_Console_Branded_organic',
       'Google_Search_Console_Non_branded_organic', 'Social_media_organic',
       'Search_organic', 'Email_organic', 'SMS_organic', 'Referral_organic',
       'Affiliate_organic', 'Email_campaigns_organic', 'SMS_flows_organic',
       'SMS_campaigns_organic', 'Email_flows_organic', 'Username'],
      dtype='object'

In [69]:
ads_columns = df_campaign_weekly_cleaned.columns[df_campaign_weekly_cleaned.columns.str.contains('Ads')].tolist()
ads_columns

['Facebook_Ads_Tests',
 'Facebook_Ads_Retargeting',
 'Facebook_Ads_Advantage_Plus',
 'TikTok_Ads_Nurturing',
 'Google_DSP_Ads_All',
 'TikTok_Ads_Prospecting',
 'Facebook_Ads_Lead_generation',
 'Google_Ads_Video',
 'Bing_Ads_Branded_search',
 'Bing_Ads_Non_branded_search',
 'Bing_Ads_Shopping',
 'Google_Ads_Demand_gen',
 'Facebook_Ads_Nurturing',
 'Google_Ads_Shopping',
 'Google_Ads_Non_branded_search',
 'Facebook_Ads_Promotions',
 'Facebook_Ads_Prospecting',
 'Google_Ads_Branded_search',
 'Google_Ads_Performance_Max',
 'TikTok_Ads_Product_ads']

In [70]:
organic_columns = df_campaign_weekly_cleaned.columns[df_campaign_weekly_cleaned.columns.str.contains('organic')].tolist()
organic_columns = [col for col in organic_columns if col not in ['Search_organic', 'Email_organic', 'SMS_organic']]
organic_columns

['Google_Search_Console_Branded_organic',
 'Google_Search_Console_Non_branded_organic',
 'Social_media_organic',
 'Referral_organic',
 'Affiliate_organic',
 'Email_campaigns_organic',
 'SMS_flows_organic',
 'SMS_campaigns_organic',
 'Email_flows_organic']

#### Map the column names to the variable types. The required variable types are **time, controls, kpi, revenue_per_kpi, media and media_spend**.

#### controls paramter is required -> GQV is one of the best choices but [Here](https://https://developers.google.com/meridian/docs/advanced-modeling/control-variables) is some theory to guide diverse options

In [71]:
coord_to_columns = load.CoordToColumns(
    time=time,
    #geo='geo',
    controls=['Promotion', 'Google_Search_Console_Branded_organic','Google_Search_Console_Non_branded_organic' ],
    #population='population',
    kpi='Sales_web',
    #revenue_per_kpi='revenue_per_conversion',
    media=ads_columns,
    media_spend=ads_columns,
    organic_media=organic_columns,
    non_media_treatments=['Promotion'],
)

In [72]:
ads_dict = {ad: ad.replace('_Ads_', ' ').replace('_Ads', '').replace('_', ' ') for ad in ads_columns}
ads_dict

{'Facebook_Ads_Tests': 'Facebook Tests',
 'Facebook_Ads_Retargeting': 'Facebook Retargeting',
 'Facebook_Ads_Advantage_Plus': 'Facebook Advantage Plus',
 'TikTok_Ads_Nurturing': 'TikTok Nurturing',
 'Google_DSP_Ads_All': 'Google DSP All',
 'TikTok_Ads_Prospecting': 'TikTok Prospecting',
 'Facebook_Ads_Lead_generation': 'Facebook Lead generation',
 'Google_Ads_Video': 'Google Video',
 'Bing_Ads_Branded_search': 'Bing Branded search',
 'Bing_Ads_Non_branded_search': 'Bing Non branded search',
 'Bing_Ads_Shopping': 'Bing Shopping',
 'Google_Ads_Demand_gen': 'Google Demand gen',
 'Facebook_Ads_Nurturing': 'Facebook Nurturing',
 'Google_Ads_Shopping': 'Google Shopping',
 'Google_Ads_Non_branded_search': 'Google Non branded search',
 'Facebook_Ads_Promotions': 'Facebook Promotions',
 'Facebook_Ads_Prospecting': 'Facebook Prospecting',
 'Google_Ads_Branded_search': 'Google Branded search',
 'Google_Ads_Performance_Max': 'Google Performance Max',
 'TikTok_Ads_Product_ads': 'TikTok Product ads'

In [73]:
correct_media_to_channel = ads_dict
correct_media_spend_to_channel = ads_dict

In [77]:
df_campaign_weekly_cleaned.index

DatetimeIndex(['2022-11-13', '2022-11-20', '2022-11-27', '2022-12-04',
               '2022-12-11', '2022-12-18', '2022-12-25', '2023-01-01',
               '2023-01-08', '2023-01-15',
               ...
               '2024-12-08', '2024-12-15', '2024-12-22', '2024-12-29',
               '2025-01-05', '2025-01-12', '2025-01-19', '2025-01-26',
               '2025-02-02', '2025-02-09'],
              dtype='datetime64[ns]', name='Date', length=118, freq='W-SUN')

#### Datetime Index error
#### we may need to generate GVQ and brainstorm which variables are control variables/confounder variables

In [74]:
loader = load.DataFrameDataLoader(df=df_campaign_weekly_cleaned, kpi_type= 'Sales_web', coord_to_columns=coord_to_columns, media_to_channel=correct_media_to_channel, media_spend_to_channel=correct_media_spend_to_channel)

KeyError: "None of [DatetimeIndex(['2020-03-03', '2020-05-10', '2020-06-29', '2020-07-27',\n               '2020-09-27', '2021-04-22', '2021-06-21', '2021-11-06',\n               '2021-11-07', '2021-12-05',\n               ...\n               '2024-05-21', '2024-06-01', '2024-06-06', '2024-06-29',\n               '2024-08-03', '2024-08-11', '2024-08-16', '2024-10-10',\n               '2025-01-27', '2022-10-13'],\n              dtype='datetime64[ns]', length=1640, freq=None)] are in the [index]"

In [ ]:
loader = load.CsvDataLoader(
    csv_path="/Users/fcremer29/focus/meridian_test/meridian/data/simulated_data/csv/geo_all_channels.csv", #can't use relative path idk why
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()